<a href="https://colab.research.google.com/github/Kxshxv/invoice_extractor/blob/main/invoice_extracter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install -q -U google-generativeai


In [27]:
import google.generativeai as genai

seting and api key

In [28]:
from google.colab import userdata
google_api_key = userdata.get('google_api_key')
google_api_key = google_api_key.strip().replace('\r', '').replace('\n', '')
genai.configure(api_key=google_api_key)

listing of the models available

In [29]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/learnlm-1.5-pro-experimental
models/gemini-exp-1114
models/gemini-exp-1121


In [30]:
MODEL_CONFIG = {
    "temperature": 0.2,
    "top_p": 1,
    "top_k": 32,
    "max_output_tokens": 4096,
}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"}
]



In [35]:
model = genai.GenerativeModel(model_name="gemini-1.5-flash", generation_config= MODEL_CONFIG, safety_settings= safety_settings)

define image format to input in gemini


In [36]:
from pathlib import Path

def image_formart(image_path):
  image_path = Path(image_path)

  if not image_path.exists():
    raise FileNotFoundError(f"Could not find image: {image_path}")

  image_parts = [
      {
          "mime_type": "image/png",
          "data": image_path.read_bytes()
      }
  ]
  return image_parts


gemini model output(user prompt)

In [37]:
def gemini_output(image_path, system_prompt, user_prompt):
  image_info = image_formart(image_path)
  input_prompt = [ system_prompt, image_info[0], user_prompt]
  response = model.generate_content(input_prompt)
  return response.text

extracting part of information

In [39]:
system_prompt = " exctact the data from the invoice"

image_path = "/content/invoice.png"

user_prompt= "what is the GSTIN number"

gemini_output(image_path, system_prompt, user_prompt)

'The GSTIN number for the seller is 29AAACD0474C1ZI.  The GSTIN number for the buyer is 29AABCA8887F1Z8.'

extractind all the data

In [47]:
system_prompt = " extract all the data from the invoice"

image_path = "/content/invoice.png"

user_prompt= "Convert Invoice data into json format with appropriate json tags as required for the data in image"



In [45]:
output =gemini_output(image_path, system_prompt, user_prompt)

In [46]:
from IPython.display import Markdown
Markdown(output)

```json
{
  "invoice_number": "IN2912407402",
  "invoice_date": "14.06.2024",
  "due_date": "15.07.2024",
  "company": "DABUR INDIA LIMITED",
  "company_address": "Regd Office:8/3, Asaf Ali Road, New Delhi 110 002",
  "company_gst": "29AAACD0474C1ZI",
  "company_pan": "AAACD0474C",
  "company_cin": "L24230DL1975PLC007908",
  "company_phone": "011-23253488",
  "company_email": "Corpcomm@dabur.com",
  "billing_address": {
    "name": "ATLAS DISTRIBUTORS PVT.LTD",
    "address": "No.1060/1047/941/1/77/1A/33/2, Eswari Industrial Estate Hulimavu Main R, Bannerghatta Road,, Bangalore, - 560076",
    "pan": "AABCA8887F",
    "gst": "29AABCA8887F1Z8"
  },
  "shipping_address": {
    "name": "ATLAS DISTRIBUTORS PVT.LTD",
    "address": "No.1060/1047/941/1/77/1A/33/2, Eswari Industrial Estate Hulimavu Main R, Bannerghatta Road,, Bangalore, - 560076"
  },
  "items": [
    {
      "item_code": "FF010180R",
      "product_description": "Real Fruit Power Mango 180ml",
      "hsn_code": "22029920",
      "mfg_date": "04/24",
      "exp_date": "11/24",
      "batch": "RU0228",
      "unit_per_case": "30",
      "mrp": "20.00",
      "rate_per_unit": "11.05",
      "qty_cases": "20",
      "qty_units": "600",
      "uom": "EA",
      "free_qty_cases": "0",
      "free_qty_units": "0",
      "gross_amt": "6,630.13",
      "disc_percent": "0",
      "disc_amt": "0",
      "taxable_amt": "6630.13",
      "total_gst": "795.62",
      "total_gst_percent": "12.00",
      "cess_percent": "0.00",
      "cess_amt": "0.00",
      "invoice_amt": "7425.75"
    },
    {
      "item_code": "FF0101LTR",
      "product_description": "Real Fruit Power Mango - 1 LTR",
      "hsn_code": "22029920",
      "mfg_date": "05/24",
      "exp_date": "12/24",
      "batch": "ID00007",
      "unit_per_case": "12",
      "mrp": "115.00",
      "rate_per_unit": "71.20",
      "qty_cases": "50",
      "qty_units": "600",
      "uom": "EA",
      "free_qty_cases": "0",
      "free_qty_units": "0",
      "gross_amt": "42,717.47",
      "disc_percent": "0",
      "disc_amt": "0",
      "taxable_amt": "42717.47",
      "total_gst": "5126.1",
      "total_gst_percent": "12.00",
      "cess_percent": "0.00",
      "cess_amt": "0.00",
      "invoice_amt": "47843.57"
    },
    // ... (rest of the items)
  ],
  "total": {
    "gross_amt": "351107.07",
    "total_gst": "42132.84",
    "total_cess": "0.00",
    "invoice_amt": "393239.91"
  },
  "payment_terms": "Payment due after 30 days from the date of invoice"
}
```